# 02_ocr_testing.ipynb
## (Refactorizado) Paso 2: Extracción con API ADE

**Objetivo:** Probar el endpoint `v1/ade/parse` de Landing AI para extraer Markdown estructurado de:
1. Los **PDFs originales** (de `data/input/pdf`)
2. Las **imágenes de Excel** (de `data/images/excel_images`)

In [1]:
import sys
import logging
from pathlib import Path
import json
import os
import requests # Necesario para las llamadas a la API

# --- Celda de Configuración Estándar ---
PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

import src.utils.config 
from src.utils.config import (
    LANDING_AI_API_KEY, # Importamos la clave
    INPUT_PDF_DIR,      
    EXCEL_IMAGES_DIR,   
    EXTRACTED_TEXT_DIR  
)

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print(f"Raíz del proyecto establecida en: {PROJECT_ROOT}")
print("✅ Entorno configurado. Logging listo.")

Raíz del proyecto establecida en: c:\Users\Usuario\Documents\UTEC\Liquidaciones Agent\multidoc-agent
✅ Entorno configurado. Logging listo.


## 1. Funciones de Prueba de API (Locales)

In [2]:
# --- Configuración de API Key ---
# Usamos la clave importada de config.py
MI_API_KEY = LANDING_AI_API_KEY
ADE_API_URL = "https://api.va.landing.ai/v1/ade/parse"

if not MI_API_KEY or "..." in MI_API_KEY:
    logger.error("¡ADVERTENCIA! Reemplaza '...' en tu .env con tu API Key de Landing AI.")
    # Levantamos un error para detener la ejecución si la clave falta
    raise ValueError("Falta LANDING_AI_API_KEY. Revisa tu archivo .env")
else:
    print("API Key de Landing AI cargada.")

def extract_ade_from_pdf(pdf_path: Path) -> dict:
    """Envía un PDF COMPLETO al endpoint ADE."""
    logger.info(f"Procesando PDF: {pdf_path.name} con ADE...")
    headers = {"Authorization": f"Bearer {MI_API_KEY}"}
    if not pdf_path.exists():
        return {"status": "error", "message": "Archivo no encontrado"}

    with open(pdf_path, 'rb') as f_pdf:
        files_payload = {'document': (pdf_path.name, f_pdf, 'application/pdf'), 'model': (None, 'dpt-2-latest')}
        try:
            response = requests.post(ADE_API_URL, headers=headers, files=files_payload, timeout=120)
            response.raise_for_status()
            return {"status": "success", "provider": "landing_ai_ade_pdf", "source_file": pdf_path.name, "text": response.json().get('markdown', '')}
        except requests.exceptions.RequestException as e:
            logger.error(f"Error de API (PDF) para {pdf_path.name}: {e}")
            return {"status": "error", "message": str(e), "source_file": pdf_path.name}

def extract_ade_from_image(image_path: Path) -> dict:
    """Envía una IMAGEN (PNG) al endpoint ADE."""
    logger.info(f"Procesando Imagen: {image_path.name} con ADE...")
    headers = {"Authorization": f"Bearer {MI_API_KEY}"}
    if not image_path.exists():
        return {"status": "error", "message": "Archivo no encontrado"}

    with open(image_path, 'rb') as f_img:
        files_payload = {'document': (image_path.name, f_img, 'image/png'), 'model': (None, 'dpt-2-latest')}
        try:
            response = requests.post(ADE_API_URL, headers=headers, files=files_payload, timeout=60)
            response.raise_for_status()
            return {"status": "success", "provider": "landing_ai_ade_image", "source_file": image_path.name, "text": response.json().get('markdown', '')}
        except requests.exceptions.RequestException as e:
            logger.error(f"Error de API (Imagen) para {image_path.name}: {e}")
            return {"status": "error", "message": str(e), "source_file": image_path.name}

print("Funciones 'extract_ade_from_pdf' y 'extract_ade_from_image' definidas para prueba.")

API Key de Landing AI cargada.
Funciones 'extract_ade_from_pdf' y 'extract_ade_from_image' definidas para prueba.


## 2. Ejecutar Extracción (Simulando Paso 2 del pipeline)

In [5]:
# Listar todos los archivos de entrada que vamos a procesar
pdf_files = list(INPUT_PDF_DIR.glob("*.pdf"))
excel_png_files = list(EXCEL_IMAGES_DIR.glob("*.png"))

print(f"PDFs originales encontrados: {len(pdf_files)}")
print(f"Imágenes PNG de Excel encontradas: {len(excel_png_files)}")

all_document_paths = pdf_files + excel_png_files
print(f"Total de documentos a procesar: {len(all_document_paths)}")

all_ade_results = []

PDFs originales encontrados: 1
Imágenes PNG de Excel encontradas: 8
Total de documentos a procesar: 9


In [ ]:
print("="*60)
print("INICIANDO EXTRACCIÓN DE MARKDOWN (ADE)")
print("="*60)

for i, doc_path in enumerate(all_document_paths, 1):
    
    if doc_path.suffix.lower() == ".pdf":
        result = extract_ade_from_pdf(doc_path)
        prefix = "PDF"
    else:
        result = extract_ade_from_image(doc_path)
        prefix = "Imagen"
        
    all_ade_results.append(result)
    
    if result['status'] == 'success':
        print(f"  [{i}/{len(all_document_paths)}] ✅ Éxito ({prefix}): {doc_path.name}. Caracteres: {len(result['text'])}")
    else:
        print(f"  [{i}/{len(all_document_paths)}] ❌ Error ({prefix}): {doc_path.name}. Mensaje: {result['message']}")
print("="*60)

## 3. Guardar Resultados (Simulando Pipeline)

In [6]:
print(f"\nProcesamiento completado. Total de resultados: {len(all_ade_results)}")

# Guardamos los resultados en la carpeta que usará el siguiente paso (Parser)
success_count = 0
EXTRACTED_TEXT_DIR.mkdir(parents=True, exist_ok=True) # Asegurarse de que exista

for result in all_ade_results:
    if result['status'] == 'success':
        base_name = Path(result['source_file']).stem
        output_filename = f"{base_name}_text.json"
        output_path = EXTRACTED_TEXT_DIR / output_filename
        
        try:
            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump(result, f, ensure_ascii=False, indent=2)
            success_count += 1
        except Exception as e:
            logger.error(f"Error al guardar {output_filename}: {e}")

print(f"Resultados exitosos guardados en {EXTRACTED_TEXT_DIR.relative_to(PROJECT_ROOT)}: {success_count}")


Procesamiento completado. Total de resultados: 0
Resultados exitosos guardados en data\processed\extracted_text: 0


## 4. Prueba de Parseo (Simulando Paso 3)

In [3]:
# Ahora, probemos el siguiente paso del pipeline: el StructureParser
# Importamos las clases y funciones del Paso 4
from src.extractors.structure_parser import StructureParser, LiquidacionData
from langchain_openai import ChatOpenAI
from src.utils.config import LLM_MODEL, LLM_TEMPERATURE, OPENAI_API_KEY

parser = None
if not OPENAI_API_KEY or "..." in OPENAI_API_KEY:
    print("⚠️ No se encontró OPENAI_API_KEY. Saltando prueba de parseo.")
    print("   Configúrala en tu .env si quieres probar el Paso 3.")
else:
    # Inicializar el LLM y el Parser
    llm = ChatOpenAI(model=LLM_MODEL, temperature=LLM_TEMPERATURE)
    parser = StructureParser(llm=llm, schema=LiquidacionData)
    print(f"✅ Parser (Markdown -> JSON) inicializado con modelo: {LLM_MODEL}")

c:\Users\Usuario\Documents\UTEC\Liquidaciones Agent\multidoc-agent\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-11-08 11:37:09,593 - src.extractors.structure_parser - INFO - StructureParser inicializado con el esquema: LiquidacionData


INFO:src.extractors.structure_parser:StructureParser inicializado con el esquema: LiquidacionData


✅ Parser (Markdown -> JSON) inicializado con modelo: gpt-5-nano


In [7]:
# Ejecutar el Parseo sobre el primer resultado exitoso
if parser and all_ade_results:
    # Buscar el primer resultado exitoso para probar
    first_success = next((r for r in all_ade_results if r['status'] == 'success'), None)
    
    if first_success:
        markdown_text = first_success['text']
        source_name = first_success['source_file']
        print(f"\n--- Parseando: {Path(source_name).stem}_text.json ---")
        
        try:
            structured_data = parser.parse_document(markdown_text)
            
            # Guardar el resultado (simulando el pipeline real)
            output_filename = f"{Path(source_name).stem}_structure.json"
            output_path = EXTRACTED_TEXT_DIR.parent / "extracted_tables" / output_filename
            output_path.parent.mkdir(exist_ok=True)
            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump(structured_data, f, ensure_ascii=False, indent=2)
            
            print(f"✅ ¡Parseo Exitoso! -> {output_path.name}")
            print("-"*50)
            print(f"Cliente: {structured_data.get('cliente_nombre')}")
            print(f"Factura: {structured_data.get('numero_factura')}")
            print(f"Total General: {structured_data.get('resumen_financiero', {}).get('total_general')}")
            print("-"*50)

        except Exception as e:
            print(f"❌ ¡Error en el Parseo! {e}")
else:
    print("\nParser no inicializado o no hay resultados de ADE para probar.")


Parser no inicializado o no hay resultados de ADE para probar.


## 5. Resumen y Próximos Pasos

In [ ]:
print("="*60)
print("RESUMEN: PASO 2 - EXTRACCIÓN Y PARSEO")
print("="*60)

print(f"✅ Se probó la API ADE ('v1/ade/parse') con {len(all_document_paths)} documentos.")
print(f"✅ La API acepta tanto PDFs como PNGs.")
print(f"✅ La API devuelve Markdown estructurado (¡mucho mejor que texto crudo!).")
print(f"✅ Se guardaron {success_count} archivos '_text.json' (con Markdown) en '{EXTRACTED_TEXT_DIR.relative_to(PROJECT_ROOT)}'.")
print(f"✅ Se probó el 'StructureParser' (Paso 3) para convertir el Markdown a JSON.")
if parser and success_count > 0:
    print(f"✅ Se guardó 1 archivo '_structure.json' en 'data/processed/extracted_tables'.")

print(f"\n🚀 PRÓXIMO PASO:")
print(f"   → Notebook 03: Generación de Embeddings (CLIP)")
print("="*60)